<a href="https://colab.research.google.com/github/noveoko/24nme.github.io/blob/main/metryki_miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path

pth = Path(r'/content/data.html')

In [5]:
from bs4 import BeautifulSoup as bs

html = None
with open(pth, 'r',encoding='utf-8') as f:
  html = f.read()

soup = BeautifulSoup(html, "html.parser")

events = {"deaths": [], "births": [], "marriages": [], "lists": []}

table_titles = {
    "Zgony": "deaths",
    "Urodzenia": "births",
    "Śluby": "marriages",
    "Spisy": "lists",
}

for title, event_type in table_titles.items():
    table = soup.find("h2", text=lambda t: t and title in t).find_next("table")
    rows = table.find_all("tr")
    header = [th.get_text(strip=True) for th in rows.pop(0).find_all("th")]

    for row in rows:
        event = {}
        for idx, td in enumerate(row.find_all("td")):
          try:
            event[header[idx]] = td.get_text(strip=True)
          except Exception as ee:
            continue
        events[event_type].append(event)


<ipython-input-5-5aa201f22975>:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  table = soup.find("h2", text=lambda t: t and title in t).find_next("table")


In [7]:
events.keys()

dict_keys(['deaths', 'births', 'marriages', 'lists'])

In [13]:
events['lists'][0:1]

[{'Nr Gospodarstwa': '48',
  'Nr Mężczyzny': '',
  'Nr Kobiety': '7',
  'Personalia': 'żona Cecylia z Martynoskich',
  'Wiek Mężczyzny': '',
  'Wiek Kobiety': '38',
  'Parafia': 'Horbulów',
  'Miejscowość': 'Dobryń',
  'Rok': '1790',
  'Archiwum': 'CDIAK',
  'Spisane przez': 'DM',
  'Sygnatura': '1044-1-1',
  'Strona': '',
  'Nr Skanu': '',
  'Uwagi 2': ''}]

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Your data
data = {...}  # put your dictionary here

# Convert dictionaries to pandas DataFrames
deaths = pd.DataFrame(data['deaths'])
births = pd.DataFrame(data['births'])
marriages = pd.DataFrame(data['marriages'])
lists = pd.DataFrame([data['lists']])  # 'lists' is a dict, not a list of dicts

# Add 'EventType' column to distinguish between different events
deaths['EventType'] = 'death'
births['EventType'] = 'birth'
marriages['EventType'] = 'marriage'
lists['EventType'] = 'list'

# Concatenate all DataFrames
df = pd.concat([deaths, births, marriages, lists])

# Create 'FullName' column
df['FullName'] = df['Imię'] + ' ' + df['Nazwisko']

# Initialize hashtable
people_hash = {}

# Populate hashtable
for i, row in df.iterrows():
    name = row['FullName']
    if name not in people_hash:
        people_hash[name] = [row.to_dict()]
    else:
        # Check if the same person is likely to appear in different events
        for person in people_hash[name]:
            if person['EventType'] != row['EventType']:
                if person['EventType'] in ['birth', 'death'] and row['EventType'] in ['birth', 'death']:
                    if abs(int(person['Rok']) - int(row['Rok'])) < 100:
                        people_hash[name].append(row.to_dict())
                else:
                    # Use fuzzy matching for other events
                    if fuzz.ratio(person['Miejscowość'], row['Miejscowość']) > 70:  # adjust the threshold as needed
                        people_hash[name].append(row.to_dict())
                        break
        else:
            people_hash[name] = [row.to_dict()]

# Now 'people_hash' is a hashtable where keys are names and values are lists of events related to the person with this name
